# 1.2 Synthetic Data Generation
Main logic for my generation:
1. Have a defined sentiment for all of my prompting as a baseline for generating the input and output
    - The generation is a diversed mix of zero shot and few shot prompting on the model to strike a balance between the variety of the prompts
    - Prompt is requested to generate of around 5-30 words which covers the 25% to 75% percentile to ensure a diverse but not obsecure range of data is generated
    - The direct request to generate inputs of a certain sentiment ensures the initial generation to already have a more or less accurate output
    - This is done as I realise that the model is rather weak and requires extensive handhelding in generation
2. Have a very small sample(of about 30 samples) of real life data inputs from my peers to make the data more diverse again and also provide a sense of human authenticity to it
3. Evaluation is done with a heuristic to make sure the data generated is of certain quality

## Main Logic to Generate Data

In [1]:
# Imports
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import json
import os
import pandas as pd

In [2]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B-Instruct")


### Function to generate user inputs

In [ ]:
def generate_user_input(shot, sentiment, type_of_input):
    """
    Generate user input for the model based on the type of input and sentiment.
    
    Args:
        shot (int): either 0 or 1, indicating whether to provide an example or not.
        sentiment (str): The sentiment of the headline (neutral, positive, negative).
        type_of_input (str): The type of input to generate (e.g., 'headline').
        
    Returns:
        str: The generated input string.
    """
    output = f"Generate a financial {type_of_input} with {sentiment} sentiment. "
    output += f"The {type_of_input} should be 5-30 words, factual in tone. "
    
    if shot == 0:
        output += f"Example: this is the {type_of_input} you generated"
        return output
    
    examples = {
        "neutral": "Example: A Sexist Joke Cost Ken Fisher $4 Billion in Assets. He Still Runs $121 Billion.\n",
        "positive": "Example: Western Union will be working with MercadoLibre, the South American eCommerce giant, so digital remittances can be sent in Mexico.\n",
        "negative": "Example: The app will be delisted from Apple's App Store on Oct. 5.\n"
    }
    
    output += examples.get(sentiment, "")
    return output

### Function to generate a input for the dataset

In [ ]:
def generate_synthetic_input(prompt):
    """
    Generate synthetic data using the provided prompt with the normal model and tokenizer.
    
    Args:
        prompt (list or str): The input prompt for the model.
        
    Returns:
        str: The generated text from the model.
    """
    inputs = tokenizer.apply_chat_template(
        prompt,
        tokenize=True,
        return_tensors="pt",
    )

    outputs = model.generate(
        input_ids=inputs, 
        max_new_tokens=200,
        use_cache=True,
        temperature=0.9, 
        min_p=0.1
    )
    return tokenizer.batch_decode(outputs)[0]

### Function to extract out the exact headline

In [5]:
# Utility function to clean the generated text
def extract_headline(generated_text):
    """
    Extract a clean headline from the generated text.
    
    Args:
        generated_text (str): Raw generated text from the model
        
    Returns:
        str: Clean headline without formatting
    """
    # Remove all EOT tags (in various forms)
    generated_text = re.sub(r'<\|eot(?:_id)?\|>', '', generated_text)
    
    # Look for content after assistant tag
    if "<|start_header_id|>assistant<|end_header_id|>" in generated_text:
        content = generated_text.split("<|start_header_id|>assistant<|end_header_id|>")[1].strip()
    else:
        content = generated_text
    
    # Find numbered headlines (like "1. Headline text")
    lines = content.split('\n')
    for line in lines:
        # Match lines like "1. Headline" or "1. **Headline**"
        match = re.search(r'\d+\.\s+(?:\*\*)?([^*\n]+)(?:\*\*)?', line)
        if match:
            headline = match.group(1).strip()
            # Remove any remaining special tokens
            headline = re.sub(r'<\|[^|]+\|>', '', headline)
            return headline
    
    # If no numbered headlines found, just return first non-empty line
    for line in lines:
        if line.strip() and not line.startswith("<|") and not line.startswith("Here are"):
            # Remove any remaining special tokens
            clean_line = re.sub(r'<\|[^|]+\|>', '', line.strip())
            return clean_line
    
    # Clean the content as a last resort
    clean_content = re.sub(r'<\|[^|]+\|>', '', content.strip())
    return clean_content

### Main Generation

In [ ]:
sentiments = ["neutral", "positive", "negative"]
type_of_shot = [0, 1]

synthetic_data = []
n = 0  # Counter for the number of generated headlines

for sentiment in sentiments:
    for shot in type_of_shot:
        messages = [
            {"role": "system", "content": "You are a market news generator. Generate ONLY the headline text. Do NOT include any introductions, explanations, or formatting instructions. Do NOT write phrases like 'Here's a headline' or 'Market headline:'. Just output the clean headline text directly."},
            {"role": "user", "content": generate_user_input(shot, sentiment, "headline")},
        ]

        i = 0
        
        while i < 79:
            if n > 470:
                break
            print("generating ", n)
            generated_text = generate_synthetic_input(messages)
        
            # Extract just the headline
            headline = extract_headline(generated_text)
            
            synthetic_data.append({
                    "input": headline,
                    "output": sentiment,
                    "instruction": "Base on the sentiment of the headline, classify it as neutral, positive, or negative."
                })
            print(headline)
            i += 1
            n += 1

print("Done generating lol")

generating  0
U.S. GDP Growth Slows to 2.3% in Q2, Reaching 61.4 Trillion Dollar Mark
generating  1
US Job Market Grows, Consumer Prices Rise Amidst Slow Economic Growth
Done generating lol


## Validation of Data

In [ ]:
import torch
import re
import language_tool_python
from transformers import AutoTokenizer, AutoModelForCausalLM

# 0) select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) grammar checker
tool = language_tool_python.LanguageTool("en-US")

# 2) load LM for scoring
tok_pp = AutoTokenizer.from_pretrained("unsloth/llama-3.2-1b-instruct")
model_pp = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3.2-1b-instruct",
    device_map="auto",
    torch_dtype=torch.float16,
).to(device).eval()

def perplexity(sentence):
    enc = tok_pp(sentence, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model_pp(**enc, labels=enc["input_ids"])
    return torch.exp(out.loss).item()

def is_high_quality(headline):
    # A) length heuristics
    w = headline.split()
    if len(w) < 5 or len(w) > 30:
        return False
    # B) ban URLs or stray tokens
    if re.search(r"http[s]?://|\<\|[^\|]+\|\>", headline):
        return False
    # C) grammar/spelling check
    errs = tool.check(headline)
    if len(errs) > 2:
        return False
    # D) fluency via perplexity
    ppl = perplexity(headline)
    if ppl > 100:
        return False
    return True


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [8]:
import pandas as pd

# 1) load your full synthetic set
csv_path = "../data/synthetic_data_generate.csv"
df = pd.read_csv(csv_path)

# 2) apply the quality heuristic
df["is_high_quality"] = df["input"].apply(is_high_quality)

# 3) report
total  = len(df)
passed = df["is_high_quality"].sum()
print(f"Total examples:     {total}")
print(f"High-quality pass:  {passed} ({passed/total:.1%})")

# 4) save only the clean ones
df_clean = df[df["is_high_quality"]].drop(columns="is_high_quality")
clean_path = "../data/synthetic_data_generate_clean.csv"
df_clean.to_csv(clean_path, index=False)
print(f"Saved {len(df_clean)} clean examples to {clean_path}")

AssertionError: Torch not compiled with CUDA enabled

### Saving the final data

In [ ]:
os.makedirs("data", exist_ok=True)

df = pd.DataFrame(synthetic_data)
csv_path = "../data/synthetic_data_generate.csv"

# Check if file exists and append if it does
if os.path.exists(csv_path):
    # Load existing data
    existing_df = pd.read_csv(csv_path)
    
    # Combine existing and new data
    combined_df = pd.concat([existing_df, df], ignore_index=True)
    
    # Save the combined data
    combined_df.to_csv(csv_path, index=False)
    print(f"Appended {len(df)} new entries to existing data. Total entries: {len(combined_df)}")
else:
    # If file doesn't exist yet, just save the new data
    df.to_csv(csv_path, index=False)
    print(f"Created new file with {len(df)} entries")